# Neural Networks in 5 minutes
In this notebook we will see how to implement a simple Neural Network from Bare Back code using numpy. The notation and backpropagation step is inspired in the amazing course of Andrew NG: [Neural Networks and Deep Learning](https://www.coursera.org/learn/neural-networks-deep-learning), in Coursera. It's a great material to start with deep Neural Networks.

As an overview we will be reviewing the following topics:
- Initalization
- Forward Propagation
- Error computing
- Backward Propagation
- Update parameters

## Step 1: Forward propagation
The forward propagation step is done using the activations of the previous layer and multiplying them by the weights in the current layer, and then applying the activation function.
We will use the sigmoid activation function as the main activation in all layers.
We can write the computations as:
$$
Z^{(i)} = W_i A_{i-1} + b_i \\
A^{(i)} = g(Z^{(i)})
$$
Where we have the following parameters: 
- $W_i$ weights of layer $i$
- $A_{i-1}$ activations of the previous layer
- $b_i$ bias vector of layer $i$
- $g$ activation function of the layer $i$

Let's define the sigmoid function and the forward propagation

In [1]:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def forward_neuron(weights, previous_activations, bias, g):
    Z = np.dot(weights, previous_activations) + bias
    A = g(Z)
    return Z, A

def forward_propagation(X, theta, activation, cache={}):
    previous_activation = X
    cache["activation_layer_0"] = X
    layers = len(theta)
    
    for layer in range(len(theta)):
        parameters = theta[layer]
        bias = parameters["bias_layer_" + str(layer + 1)]
        weights = parameters["weights_layer_" + str(layer + 1)]
        Z, A = forward_neuron(weights, previous_activation, bias, activation)
        cache["activation_layer_" + str(layer + 1)] = A
        cache["linear_layer_" + str(layer + 1)] = Z
        previous_activation = A
        
    return cache, previous_activation


## Step 2: Error calculation
The error calculation step is done computing the cost function which will be the average of the losses for our training examples.
We will use the binary cross entropy loss function to compute the error of the function with respect of the true labels.
The binary cross entropy loss is defined as:
$$
L(y, \hat{y}) = y\hspace{0.3em}log(\hat{y}) + (1-y)\hspace{0.3em}log(1 - \hat{y})
$$
Where we have the parameters:
- $y$ true label of our example
- $\hat{y}$ prediction of our neural network

When we compute the loss for many examples we get the cost function of our paramters with respect of our dataset.
Our cost function $J$ can be written as:
$$
J(X, Y; \theta) = \frac{1}{m} \sum_{i=1}^m L(y_i, \hat{y_i})
$$
Where we have the following elements:
- $X$ the matrix of our input examples
- $Y$ the matrix of our true labels predictions
- $\theta$ the learnable parameters of our network, i.e. the weights and biases of all the layers
- $y_i$ ith label in our training set
- $\hat{y_i}$ ith prediction of our network

As the formula can look scary it's a simple average over our examples.

Let's define our loss and cost function

In [2]:
# Loss function
def binary_cross_entropy(y, y_hat):
    return y*np.log(y_hat) + (1 - y)*np.log(1 - y_hat)

# Cost function given the predictions
def cost_function_2(y_hat, y, loss):
    return -1*np.mean(loss(y, y_hat))

# Cost function given the inputs
def cost_function(X, Y, theta, loss):
    _, Y_hat = predict(X, theta, sigmoid)
    return -1 *np.mean(loss(Y, Y_hat))

## Step 3 Back propagation
The back propagation step, takes the derivative of the learnable parameters with respect of the cost function. This is called the gradient vector and points in the direction the cost function is growing most quickly. To compute this vector we need to use the chain rule of calculus:
$$
\frac{\partial J}{\partial w_i}
$$
The first step we need to perform to make the back propagation is to get the gradient of the cost with respect to the last activation. Since we are using a sigmoid funciton we have the following derivative:
$$
\sigma(x) = \frac{1}{1+ e^{-x}}\\
\sigma'(x) = \sigma(x)(1 - \sigma(x))\\
$$
So the derivative in terms of the cost is:
$$
\frac{\partial J(y, \hat{y}; \theta)}{\partial A} = \frac{y}{\hat{y}} - \frac{1 - y}{1 - \hat{y}}
$$
To get the backward propagation of the output with respect to the paramter $w_i$ we need to compute:
$$
\frac{\partial J(y, \hat{y}; \theta)}{\partial w_i} = \Big( \frac{y}{\hat{y}} - \frac{1 - y}{1 - \hat{y}} \Big) \sigma'(x)
$$

In [3]:
def sigmoid_derivative(x):
    return sigmoid(x)*(1 - sigmoid(x))

def sigmoid_backward(activation_derivative, z):
    return activation_derivative * sigmoid_derivative(z)

def linear_gradient(linear_derivative, activation_parameters):
    previous_activation, weights, bias = activation_parameters
    m = previous_activation.shape[1]
    weights_derivative = 1/m * np.dot(linear_derivative, previous_activation.T)
    bias_derivative = 1/m * np.sum(linear_derivative, axis=1, keepdims=True)
    previous_activation_derivative = np.dot(weights.T, linear_derivative)
    return previous_activation_derivative, weights_derivative, bias_derivative

def activation_gradient(activation_derivative, linear_activation, activation_parameters):
    linear_derivative = sigmoid_backward(activation_derivative, linear_activation)
    previous_activation_derivative, weights_derivative, bias_derivative = linear_gradient(linear_derivative, activation_parameters)
    return previous_activation_derivative, weights_derivative, bias_derivative

def gradient_layer(cache, layer):
    l_name = str(layer)
    activation_derivative = cache["activation_derivative_layer_" + l_name]
    weights = cache["weights_layer_" + str(layer)]
    bias = cache["bias_layer_" + str(layer)]
    prev_activation = cache["activation_layer_" + str(layer)]
    linear_activation = cache['linear_layer_' + str(layer)]
    current_layer_parameters = (prev_activation, weights, bias)
    previous_activation_derivative, weights_derivative, bias_derivative = activation_gradient(activation_derivative, linear_activation, current_layer_parameters)
    cache["activation_derivative_layer_" + str(layer - 1)] = previous_activation_derivative
    cache["weights_derivative_layer_" + l_name] = weights_derivative
    cache["bias_derivative_layer_" + l_name] = bias_derivative
    return cache
    
def backward_propagation(y_hat, y, parameters, layers=2):
    cache = {**parameters}
    m = y_hat.shape[1]
    y = y.reshape(y_hat.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    y_hat_derivative = - (np.divide(y, y_hat) - np.divide(1 - y, 1 - y_hat)) 
    cache["activation_derivative_layer_" + str(layers)] = y_hat_derivative
    
    for layer in range(layers, 0, -1):
        cache = gradient_layer(cache, layer)
        
    return cache

## Step 4: Update parameters

To update the parameters at time $i$ we need to compute the gradient of the parameters and substract a portion of the gradient vector to the current weights:
$$
\nabla_{\theta}  = 
    \begin{bmatrix}
        \frac{\partial J}{\partial w_1} \\
        \frac{\partial J}{\partial w_2} \\
        \vdots \\
        \frac{\partial J}{\partial w_n} \\
    \end{bmatrix}\\
\theta_i = \theta_{i-1} - \alpha \nabla_{\theta} J
$$
The portion of the gradient substracted to the learnable paramters is called learning rate and it is one of the hyper parameters of the algorithm denoted by $\alpha$.

In [4]:
def update_parameters(cache, alpha, layers=2):
    for layer in range(layers):
        layer_name = str(layer + 1)
        cache["weights_layer_" + layer_name] -= alpha * cache["weights_derivative_layer_" + layer_name]
        cache["bias_layer_" + layer_name] -= alpha * cache["bias_derivative_layer_" + layer_name]
    return cache

## Step 5: Putting all together
Let's use our implemented methods to build our neural network
### Initialize parameters

In [5]:
np.random.seed(1)
np.random.seed(3)

def initialize(layers_dimensions):
    parameters = {}
    layers = len(layers_dimensions)            # number of layers in the network
    for layer in range(1, layers):
        current_layer_dim = layers_dimensions[layer]
        previous_layer_dim = layers_dimensions[layer - 1]
        parameters['weights_layer_' + str(layer)] = np.random.randn(current_layer_dim, previous_layer_dim) * 0.01
        parameters['bias_layer_' + str(layer)] = np.zeros((current_layer_dim, 1))
        
    return parameters

### Performing gradient descent algorithm

In [7]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
#Let's build a binary problem
y = (iris.target > 0)*np.ones(iris.target.shape)
cache = initialize([4,4,1])

epochs = 10000
for i in range(epochs):
    theta = []
    #Define theta
    theta.append({"weights_layer_1": cache['weights_layer_1'], "bias_layer_1": cache['bias_layer_1']})
    theta.append({"weights_layer_2": cache['weights_layer_2'], "bias_layer_2": cache['bias_layer_2']})
    #Forward propagation
    cache, output = forward_propagation(X.T, theta, sigmoid, cache)
    #Error calculation
    cost = cost_function_2(output, y, binary_cross_entropy)
    if i % 500 == 0:
        print("cost =====> " + str(cost) + " ======== epoch " + str(i))
    #Gradient compute
    cache = backward_propagation(output, y, cache)
    #Weights update
    cache = update_parameters(cache, 0.1)

cost =====> 0.6938988007243275 ======== epoch 0
cost =====> 0.6342570398826515 ======== epoch 500
cost =====> 0.632044787537711 ======== epoch 1000
cost =====> 0.6287053780357208 ======== epoch 1500
cost =====> 0.6243443468541711 ======== epoch 2000
cost =====> 0.618849243807791 ======== epoch 2500
cost =====> 0.6112436756910246 ======== epoch 3000
cost =====> 0.5991689915987959 ======== epoch 3500
cost =====> 0.5772064548656747 ======== epoch 4000
cost =====> 0.5311651263913292 ======== epoch 4500
cost =====> 0.4249370261101307 ======== epoch 5000
cost =====> 0.24841422785639106 ======== epoch 5500
cost =====> 0.135103810173985 ======== epoch 6000
cost =====> 0.09202899597047219 ======== epoch 6500
cost =====> 0.07368617364434649 ======== epoch 7000
cost =====> 0.06433509772364936 ======== epoch 7500
cost =====> 0.05893573419458658 ======== epoch 8000
cost =====> 0.05555952737763003 ======== epoch 8500
cost =====> 0.05334422207594495 ======== epoch 9000
cost =====> 0.05185550553974503

## Predict

In [8]:
theta = []
theta.append({"weights_layer_1": cache['weights_layer_1'], "bias_layer_1": cache['bias_layer_1']})
theta.append({"weights_layer_2": cache['weights_layer_2'], "bias_layer_2": cache['bias_layer_2']})
_, output = forward_propagation(X.T, theta, sigmoid, cache)
print(output)

[[0.02273728 0.00598972 0.00453463 0.00489739 0.02189242 0.27073357
  0.00678968 0.0192699  0.00211211 0.00840798 0.09345771 0.01578738
  0.00467889 0.00119507 0.20048741 0.62781245 0.09328273 0.02428298
  0.39531849 0.05854096 0.08744083 0.05102079 0.00375849 0.04168269
  0.03464637 0.01172003 0.0285716  0.03769389 0.02361791 0.00883992
  0.00914595 0.05755895 0.12173509 0.24709919 0.00891475 0.00652266
  0.04532442 0.01632597 0.00201811 0.02430433 0.01493353 0.00103579
  0.00265578 0.04002564 0.18879501 0.00520806 0.07214191 0.00464014
  0.07287115 0.01242257 0.99998805 0.99995431 0.99998916 0.98261906
  0.99994177 0.99944395 0.99997053 0.40994109 0.99995475 0.97975395
  0.4022209  0.99955687 0.99374142 0.99990538 0.98919194 0.99996049
  0.99959661 0.99766351 0.99951683 0.98721365 0.99994658 0.9991438
  0.99992631 0.99987109 0.9998598  0.99994352 0.99997681 0.9999872
  0.99981487 0.96828797 0.96801312 0.95077124 0.99616349 0.99994235
  0.99931853 0.99993118 0.99997927 0.99953596 0.99

In [9]:
predictions = (output > 0.5) * np.ones(y.shape)

In [10]:
predictions

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.]])

## Compute accuracy

In [11]:
correct = (predictions == y) * np.ones(y.shape)
print(correct)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
  1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1.]]


In [12]:
print(np.sum(correct)/len(correct[0]))

0.9666666666666667
